In [1]:

import os
import sys
import pinecone

currentdir = os.getcwd()

parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

import components.pinecone_langchain as plc


import scrapeandindex.sparsevectors as sv
import scrapeandindex.query as query_sql

/Users/juliev/miniforge3/envs/metrics/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


running on cpu


In [2]:
pinecone_key = os.getenv('PINECONE_API_KEY')
pinecone_env = os.getenv('PINECONE_ENV')
pinecone.init(api_key=pinecone_key, environment=pinecone_env)

In [3]:
pinecone.list_indexes()

['pinehack']

In [4]:
info = pinecone.Index('pinehack')

In [5]:
info

In [6]:
info.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'langchaindocs': {'vector_count': 1273}},
 'total_vector_count': 1273}

In [7]:
pilang = plc.LangChainPineconeClient()

In [8]:
pilang.get_relevant_text("How do I ask questions over documents using LangChain?", "LangChain")

AIMessage(content='LangChain is the subject of the document.', additional_kwargs={})

In [9]:
pilang.get_relevant_pinecone_data("How do I ask questions over documents using LangChain?")

'Document Question Answering\u200b Question answering involves fetching multiple documents, and then asking a question of them.\nThe LLM response will contain the answer to your question, based on the content of the documents. The recommended way to get started using a question answering chain is: from langchain . chains . question_answering import load_qa_chain chain = load_qa_chain ( llm , chain_type = "stuff" ) chain . run ( input_documents = docs , question = query ) The following resources exist: Question Answering Notebook: A notebook walking through how to accomplish this task. VectorDB Question Answering Notebook: A notebook walking through how to do question answering over a vector database. This can often be useful for when you have a LOT of documents, and you don\'t want to pass them all to the LLM, but rather first want to do some semantic search over embeddings.'

In [11]:
pilang.index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'langchaindocs': {'vector_count': 1273}},
 'total_vector_count': 1273}

In [12]:
embedded_query = pilang.embed.embed_query("How do I ask questions over documents using LangChain?")

In [13]:
result = pilang.index.query(
namespace='langchaindocs',
top_k=1,
vector=embedded_query,
sparse_vector = sv.get_sparse_vector("How do I ask questions over documents using LangChain?"))

In [14]:
match_id = result['matches'][0]['id']

In [15]:
import psycopg2
DATABASE_URL = os.environ['DATABASE_URL']
conn = psycopg2.connect(DATABASE_URL)
cur = conn.cursor()

In [16]:
entry = query_sql.get_heading_by_rowid('875899727194390529', cur)

In [17]:
entry[5]

'Document Question Answering\u200b Question answering involves fetching multiple documents, and then asking a question of them.\nThe LLM response will contain the answer to your question, based on the content of the documents. The recommended way to get started using a question answering chain is: from langchain . chains . question_answering import load_qa_chain chain = load_qa_chain ( llm , chain_type = "stuff" ) chain . run ( input_documents = docs , question = query ) The following resources exist: Question Answering Notebook: A notebook walking through how to accomplish this task. VectorDB Question Answering Notebook: A notebook walking through how to do question answering over a vector database. This can often be useful for when you have a LOT of documents, and you don\'t want to pass them all to the LLM, but rather first want to do some semantic search over embeddings.'

'875899727194390529'